# 1. Preprocessing

In [5]:
import numpy as np
import pandas as pd

In [6]:
from data_preprocessing import data_preprocessing

account_booking_train, external_parties_train = data_preprocessing(
    "account_booking_train.csv", "external_parties_train.csv"
)
account_booking_test, external_parties_test = data_preprocessing(
    "account_booking_test.csv", "external_parties_test.csv"
)

Loading data
Changing debit credit to 0 and 1
Loading external parties
Changing BENE to 0 and ORG to 1
Cleaning text data
Removing common words
Loading data
Changing debit credit to 0 and 1
Loading external parties
Changing BENE to 0 and ORG to 1
Cleaning text data
Removing common words


# 2. Feature Engineering

In [7]:
external_parties_train.head()

,transaction_reference_id,party_role,party_info_unstructured,parsed_name,parsed_address_street_name,parsed_address_street_number,parsed_address_unit,parsed_address_postal_code,parsed_address_city,parsed_address_state,parsed_address_country,party_iban,party_phone,external_id
0,04ff0d1c680189e3a80c92d86407f0f5,0,mary mith 107 107 angela brooks n. thomasfurt ...,mary mith,angela brooks,107 107,NaN,NaN,n. thomasfurt,NaN,bulgaria,GB49MYOB82127728573340,+1.815660-6791x8486,50039037
1,439ab0ad7380e6135ab2ff3fddd4a727,1,yesneia kim north michael 93971 koribati,yesneia kim,north michael,NaN,NaN,93971,koribati,NaN,NaN,NaN,0 (269)620-8734x2349,60044692
2,00cac12d41191a84f9e31aa731a83512,1,w. roberson jr. 41010 rachel crossingapt. 923 ...,w roberson,rachel crossingapt,41010 923,NaN,p2235417,thompsonshire amyport,NaN,NaN,GB08OTHR53515837682953,NaN,30008244
3,e4fba5f878dd3453e35973605a783a16,0,azquez-nelson co. suarez ports suite & 024 bri...,azquez nelson co,ports suite,NaN,NaN,NaN,brittanyberg,NaN,bulgaria bulgaria,GB17VVGW66321494633280,NaN,40017944
4,d03d7e4c31878b0255d39e8c3f0ab625,1,m.j. bytd iii 856 john lake s. glenn cocos (ke...,mj bytd,john lake s glenn,856,NaN,125838276,cocos (keeling),islands,NaN,NaN,(260)3371534,40012658


In [8]:
import feature_engineering

# Apply feature engineering to training data
external_parties_train = feature_engineering.perform_feature_engineering(
    external_parties_train
)

# Apply feature engineering to test data
external_parties_test = feature_engineering.perform_feature_engineering(
    external_parties_test
)

In [9]:
external_parties_train.head(15).to_csv("example.csv", index=False)

In [11]:
external_parties_train.to_csv("data/processed/external_parties_train.csv", index=False)
external_parties_test.to_csv("data/processed/external_parties_test.csv", index=False)